In [1]:
def get_cube_inertia_matrix(mass, x, y, z):
    """Given mass and dimensions of a cube return intertia matrix.
    :return: ixx, ixy, ixz, ixy, iyy, iyz, ixz, iyz, izz
    From https://www.wolframalpha.com/input/?i=moment+of+inertia+cube"""
    ixx = (1.0 / 12.0) * (y**2 + z**2) * mass
    iyy = (1.0 / 12.0) * (x**2 + z**2) * mass
    izz = (1.0 / 12.0) * (x**2 + y**2) * mass
    ixy = 0.0
    ixz = 0.0
    iyz = 0.0
    return [[ixx, ixy, ixz], [ixy, iyy, iyz], [ixz, iyz, izz]]

# get_cube_inertia_matrix(mass=0.3, x=0.15, y=0.25, z=0.05)
get_cube_inertia_matrix(mass=0.1, x=0.07, y=0.07, z=0.05)

[[6.166666666666667e-05, 0.0, 0.0],
 [0.0, 6.166666666666667e-05, 0.0],
 [0.0, 0.0, 8.166666666666668e-05]]

In [2]:
def get_cylinder_inertia_matrix(mass, r, h):
    """Given mass and dimensions of a cylinder return intertia matrix.
    :return: ixx, ixy, ixz, ixy, iyy, iyz, ixz, iyz, izz
    From https://scienceworld.wolfram.com/physics/MomentofInertiaCylinder.html"""
    ixx = (1.0 / 12.0) * h**2 * mass + (1.0 / 4.0) * r**2 * mass
    iyy = ixx
    izz = (1.0 / 2.0) * r**2 * mass
    ixy = 0.0
    ixz = 0.0
    iyz = 0.0
    return [[ixx, ixy, ixz], [ixy, iyy, iyz], [ixz, iyz, izz]]

get_cylinder_inertia_matrix(mass=0.1, r=0.015, h=0.14)

[[0.00016895833333333335, 0.0, 0.0],
 [0.0, 0.00016895833333333335, 0.0],
 [0.0, 0.0, 1.125e-05]]

In [3]:
top_mass = (153/(20+15) * 20)/1000
get_cube_inertia_matrix(mass=top_mass, x=0.2, y=0.05, z=0.05)

[[3.642857142857143e-05, 0.0, 0.0],
 [0.0, 0.0003096428571428571, 0.0],
 [0.0, 0.0, 0.0003096428571428571]]

In [4]:
from pydrake.all import (SpatialInertia)
print(top_mass)
SpatialInertia.SolidBoxWithMass(top_mass, 0.2, 0.05, 0.05).CalcRotationalInertia().CopyToFullMatrix3()

0.08742857142857141


array([[3.64285714e-05, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.09642857e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.09642857e-04]])

In [5]:
bottom_mass = ((153/(20+15)) * 15)/1000
print(bottom_mass)
SpatialInertia.SolidBoxWithMass(bottom_mass, 0.05, 0.15, 0.05).CalcRotationalInertia().CopyToFullMatrix3()

0.06557142857142857


array([[1.36607143e-04, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.73214286e-05, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.36607143e-04]])

Here I use the an SDF with two links that are welded together each with their own inertial matrix in order to calculate the combined inertial matrix and center of mass so that I can put that in the SDF where the entire T is a single link.

In [6]:
from pydrake.all import (MultibodyPlant, MultibodyPlantConfig, DiagramBuilder, AddMultibodyPlant, ModelVisualizer)
from planning_through_contact.simulation.sim_utils import GetParser, ConfigureParser

time_step = 1e-3
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlant(
    MultibodyPlantConfig(
        time_step=time_step,
        discrete_contact_approximation="similar"),
    builder)
parser = GetParser(plant)

# slider_sdf_url = "package://planning_through_contact/box_hydroelastic.sdf"
slider_sdf_url = "package://planning_through_contact/t_pusher_separate.sdf"
parser.AddModels(url=slider_sdf_url)
plant.Finalize()

sim = builder.Build()
context = sim.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
slider_frame = plant.GetFrameByName("t_pusher_top")
body_indices = plant.GetBodyIndices(plant.GetModelInstanceByName("t_pusher"))
spatial_inertia = plant.CalcSpatialInertia(plant_context, slider_frame, body_indices)

mass = spatial_inertia.get_mass()
center_of_mass = spatial_inertia.get_com()
matrix6 = spatial_inertia.CopyToFullMatrix6()

print(f"mass = {mass}\n")
print(f"p_PScm = center of mass = {center_of_mass}\n")
print(f"I_SP = rotational inertia = \n{matrix6[:3, :3]}\n")

mass = 0.15299999999999997

p_PScm = center of mass = [ 0.         -0.04285714  0.        ]

I_SP = rotational inertia = 
[[0.00082875 0.         0.        ]
 [0.         0.00033696 0.        ]
 [0.         0.         0.00110196]]



In [7]:
from pydrake.geometry import StartMeshcat

# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


Use the following cell to visualize the inertias of the model, both the split sdf and the combined one.

In [12]:
slider_sdf_url = "package://planning_through_contact/t_pusher.sdf"
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
vis_parser = visualizer.parser()
ConfigureParser(vis_parser)
vis_parser.AddModels(url=slider_sdf_url)
visualizer.Run(loop_once=True)